In [1]:
import pandas as pd
import requests
import zipfile
from pathlib import Path
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from typing import Optional
import warnings
warnings.filterwarnings("ignore")

def expand_list_column(
    df: pd.DataFrame,
    list_col: str,
    prefix: str = "v"
) -> pd.DataFrame:
    """
    list_col içindeki listeleri ayrı sütunlara açar.
    Örn: [35.18, 14.64, 0] → v1, v2, v3
    """
    values = df[list_col].apply(pd.Series)
    values.columns = [f"{prefix}{i+1}" for i in range(values.shape[1])]
    return pd.concat([df.drop(columns=[list_col]), values], axis=1)




def fetch_lbma_price(
    url: str = None,
    session: Optional[requests.Session] = None,
    price_col_idx: int = 1,           # v1 = USD fiyatı (1. eleman)
    list_col: str = "v",
    date_col: str = "d",
    value_name: str = "gold_valuation_price",
) -> pd.DataFrame:
    """
    LBMA JSON → tidy DataFrame
    - price_col_idx: v1, v2, v3... için 1 tabanlı index
    - value_name: çıktı kolon adı
    """
    s = session or requests.Session()
    data = s.get(url, params={"r": 155972931}).json()

    return (
        pd.DataFrame(data)
        .pipe(expand_list_column, list_col=list_col, prefix="v")
        .assign(
            data_date=lambda d: pd.to_datetime(d[date_col])
        )
        .rename(
            columns={f"v{price_col_idx}": value_name}
        )
        .loc[:, ["data_date", value_name]]
        .set_index("data_date")
        .sort_index()
    )


def get_download_dir(dirname: str = "data") -> Path:
    """
    Mevcut çalışma dizini (Path.cwd()) altındaki data klasörünü hazırlar.
    Eski zip/xls dosyalarını siler.
    """
    download_dir = Path.cwd() / dirname
    download_dir.mkdir(exist_ok=True)

    # Eski zip/xls temizliği
    for f in download_dir.glob("*.zip"):
        f.unlink()
    for f in download_dir.glob("*.xls*"):
        f.unlink()

    return download_dir


def fetch_latest_urdl_zip(download_dir: Path) -> Path:
    """
    TCMB URDL sayfasından en güncel haftalık URDL ZIP dosyasını indirir.
    """
    resp = requests.get(TCMB_URDL_URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Önce class'a göre ara
    zip_tag = soup.select_one("a.zip.type-link")
    # Bulamazsa href içinde .zip geçen linki ara
    if zip_tag is None:
        zip_tag = soup.find("a", href=lambda h: h and ".zip" in h)
    if zip_tag is None:
        raise RuntimeError("Sayfada .zip linki bulunamadı.")

    zip_url = urljoin(TCMB_URDL_URL, zip_tag["href"])
    zip_path = download_dir / "URDL_latest.zip"

    with requests.get(zip_url, stream=True) as r:
        r.raise_for_status()
        with zip_path.open("wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

    return zip_path


def extract_zip(zip_path: Path, extract_dir: Path) -> None:
    """
    ZIP dosyasını verilen klasöre açar.
    """
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)


def find_urdl_excel(download_dir: Path) -> Path:
    """
    data klasöründe URDL geçen ilk Excel dosyasını bulur.
    """
    excel_files = list(download_dir.glob("*URDL*.xls*"))
    if not excel_files:
        raise FileNotFoundError(f"{download_dir} içinde 'URDL' içeren Excel bulunamadı.")
    # Tek olmak zorunda zaten
    return excel_files[0]


def parse_urdl_excel_to_long(excel_path: Path) -> pd.DataFrame:
    """
    URDL Excel'ini tidy (long) formata çevirir:
      - boş satır/sütunları temizler
      - header satırını programatik seçer (tarihlerin en yoğun olduğu satır)
      - ilk kolon 'item', ikinci kolon 'aylık' yapılır
      - geniş tablo melt edilerek Tarih-altin_swap_milyar_usd long formatına getirilir
    """
    # Ham okuma, header yok
    df = pd.read_excel(excel_path, header=None)

    # Tamamen boş satır ve sütunları at
    df = df.dropna(how="all", axis=0).dropna(how="all", axis=1).reset_index(drop=True)

    # Header satırını programatik bul:
    #  - 0. kolonda item isimleri var
    #  - 1. kolonda 'Aylık' vb metin var
    #  - 2. kolondan itibaren tarihler var
    # Bu yüzden sadece 2: sütun aralığına bakıyoruz.
    date_like_counts = []
    max_check_rows = min(10, len(df))  # ilk 10 satır yeterli
    for i in range(max_check_rows):
        ser = pd.to_datetime(df.iloc[i, 2:], errors="coerce", dayfirst=True)
        date_like_counts.append(ser.notna().sum())

    # En çok tarih-like değer içeren satır header olsun
    header_row = max(range(max_check_rows), key=lambda i: date_like_counts[i])

    header = df.iloc[header_row].copy()
    # 2. sütundan sonrası tarih
    header.iloc[2:] = pd.to_datetime(
        header.iloc[2:],
        errors="coerce",
        dayfirst=True
    ).dt.date

    # Dataframe'i header'dan sonrası olarak al
    df = df.iloc[header_row + 1:].reset_index(drop=True)
    df.columns = header

    # Tamamen boş satır kalırsa temizle
    df = df.dropna(how="all")

    # Kolon isimlerini düzenle: ilk iki kolon label, geri kalanı tarihler
    cols = df.columns.to_list()
    cols[0] = "item"
    cols[1] = "aylık"
    df.columns = cols

    # Tidy (long) format
    df_long = (
        df
        .drop(columns="aylık")
        .melt(id_vars="item", var_name="Tarih", value_name="altin_swap_milyar_usd")
        .dropna(subset=["altin_swap_milyar_usd"])
    )

    # Tarih kolonunu datetime'a çevir ve index yap
    df_long["Tarih"] = pd.to_datetime(df_long["Tarih"])
    df_long = df_long.set_index("Tarih").sort_index()

    return df_long


def get_altin_swap_series(df_urdl_long: pd.DataFrame) -> pd.DataFrame:
    """
    URDL long dataframe içinden 'II.3. Diğer' satırlarına karşılık gelen
    altın swap (milyar USD) serisini döndürür.
    """
    altin_swap = (
        df_urdl_long[df_urdl_long["item"] == "II.3. Diğer"]
        .drop(columns="item")
        .rename(columns={"altin_swap_milyar_usd": "altin_swap_milyar_usd"})
    )
    return altin_swap


def fetch_altin_swap_from_urdl() -> pd.DataFrame:
    """
    FULL PIPELINE:
      - current folder altındaki data/ klasörünü hazırlar
      - TCMB URDL sayfasından son haftalık ZIP'i indirir
      - ZIP'i data/ altına açar
      - URDL Excel'ini bulur
      - tidy long dataframe üretir
      - 'II.3. Diğer' satırından altın swap serisini döndürür
    """
    download_dir = get_download_dir(dirname="data")
    zip_path = fetch_latest_urdl_zip(download_dir)
    extract_zip(zip_path, download_dir)
    excel_path = find_urdl_excel(download_dir)
    df_long = parse_urdl_excel_to_long(excel_path)
    altin_swap = get_altin_swap_series(df_long)
    return altin_swap


def evds_serie_market_all(
    series_code: str,
    start: str,
    end: str,
    datagroup: str = "",
    category: str = "",
    idx: int = 0,
    page_size: int = 20,
    aggregation_type: str = "last",
    frequency: str = "YEARWEEK",
    date_format_value: Optional[str] = None,
    verbose: bool = False,
    session: Optional[requests.Session] = None,
) -> pd.DataFrame:
    
    s = session or requests.Session()
    s.headers.update({
        "Accept": "*/*",
        "Accept-Language": "tr-TR,tr;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "User-Agent": "Mozilla/5.0",
        "X-Requested-With": "XMLHttpRequest",
        "Referer": "https://evds2.tcmb.gov.tr/index.php?/evds/serieMarket",
        "Origin": "https://evds2.tcmb.gov.tr",
        "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    })

    try:
        g = s.get("https://evds2.tcmb.gov.tr/index.php?/evds/serieMarket", timeout=30)
        if verbose:
            print("GET status:", g.status_code)
    except requests.RequestException as e:
        if verbose:
            print(f"GET isteğinde hata: {e}")
        return pd.DataFrame()

    select_code = f"{series_code}-{idx}"
    if date_format_value is None:
        date_format_value = "yyyy-ww"

    all_rows = []
    total_count = None
    skip = 0

    while True:
        payload = {
            "orderby": "Tarih desc",
            "thousand": 1,
            "decimal": 2,
            "sort": "Tarih#true",
            "frequency": frequency,
            "aggregationType": aggregation_type,
            "formula": 0,
            "graphicType": "0",
            "skip": skip,
            "take": page_size,
            "select": select_code,
            "startDate": start,
            "endDate": end,
            "categories": category,
            "mongoAdresses": "evds",
            "datagroupString": datagroup,
            "obsCountEnabled": "",
            "obsCount": "",
            "userId": "",
            "dateFormatValue": date_format_value,
            "customFormula": "null",
            "excludedSeries": "null",
        }

        try:
            r = s.post("https://evds2.tcmb.gov.tr/EVDSServlet", data=payload, timeout=60)
            if verbose:
                print(f"skip={skip} | status={r.status_code}")
            r.raise_for_status()
        except requests.RequestException as e:
            if verbose:
                print(f"POST isteğinde hata (skip={skip}): {e}")
            break

        try:
            data = r.json()
        except ValueError:
            break

        if isinstance(data, list) and data:
            if total_count is None:
                total_count = data[0].get("totalCount", len(data))
            items_list = [row.get("items", {}) for row in data]
        elif isinstance(data, dict) and "items" in data:
            if total_count is None:
                total_count = data.get("totalCount", len(data["items"]))
            items_list = data["items"]
        else:
            items_list = []

        if not items_list:
            break

        all_rows.extend(items_list)
        got = len(items_list)
        skip += got

        if total_count is not None and len(all_rows) >= total_count:
            break

    if not all_rows:
        return pd.DataFrame()

    df = pd.DataFrame(all_rows)

    # Tarih kolonlarını parse et
    for col in df.columns:
        cl = col.lower()
        if cl.startswith("tar") or cl.startswith("date"):
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

    # YEARWEEK -> Tarih (Pazartesi)
    if "YEARWEEK" in df.columns and "Tarih" not in df.columns:
        def yearweek_to_date(val):
            if pd.isna(val):
                return pd.NaT
            s = str(int(val)).zfill(6)
            year = int(s[:4])
            week = int(s[4:])
            try:
                return pd.to_datetime(f"{year}-W{week}-1", format="%G-W%V-%u")
            except Exception:
                return pd.NaT

        df["Tarih"] = df["YEARWEEK"].apply(yearweek_to_date)

    if "Tarih" in df.columns:
        df = df.sort_values("Tarih").reset_index(drop=True)

    return df


def fetch_single_series(
    series_code: str,
    col_name: str,
    start: str,
    end: str,
    frequency: str,
    date_format_value: str,
    verbose: bool = False,
    session: Optional[requests.Session] = None,
    raise_if_empty: bool = True,
) -> pd.DataFrame:
    
    df = evds_serie_market_all(
        series_code=series_code,
        start=start,
        end=end,
        frequency=frequency,
        date_format_value=date_format_value,
        verbose=verbose,
        session=session,
    )

    if df.empty:
        if raise_if_empty:
            raise ValueError(f"{series_code} için veri gelmedi.")
        out = pd.DataFrame(columns=[col_name])
        out.index = pd.DatetimeIndex([], name="Tarih")
        return out

    if "Tarih" not in df.columns:
        raise ValueError(f"{series_code} için 'Tarih' yok. Kolonlar: {df.columns.tolist()}")

    non_value_cols = {"Tarih", "YEARWEEK", "UNIXTIME", "totalCount"}
    value_cols = [c for c in df.columns if c not in non_value_cols]

    if len(value_cols) != 1:
        raise ValueError(
            f"{series_code} için beklenmeyen kolon yapısı: {df.columns.tolist()}"
        )

    val_col = value_cols[0]

    out = (
        df[["Tarih", val_col]]
        .rename(columns={val_col: col_name})
        .set_index("Tarih")
        .sort_index()
    )
    return out


# =============================================================================
# 2) Haftalık resmi altın verileri
# =============================================================================

SERIES_CONFIG = [
    ("TP.DK.USD.A.YTL", "usdtry_weekly"),
    ("TP.AB.C1",        "altin_rezerv_milyar_usd"),
    ("TP.BL002",        "a11"),
    ("TP.BL0021",       "a11_gram"),
    ("TP.BL0451",       "a616_safi_olmayan_gram"),
    ("TP.BL128",        "bl128"),
    ("TP.BL137",        "yurt_ici_mevduat_altin"),
    ("TP.BL0823",       "hazine_altin"),
    ("TP.BL0891",       "zorunlu_karsilik_altin"),
    ("TP.BL142",        "yurt_disi_bankalar_altin"),
    ("TP.BL1111",       "bl111"),
]


def fetch_weekly_official_from_evds(
    start: str = None ,
    end: Optional[str] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Yukarıdaki SERIES_CONFIG listesindeki tüm serileri YEARWEEK frekansta çeker
    ve tek DataFrame'de birleştirir.
    """

    if end is None:
        end = datetime.today().strftime("%d-%m-%Y")

    session = requests.Session()
    frequency = "YEARWEEK"
    date_format_value = "yyyy-MM-dd"

    common_kwargs = dict(
        start=start,
        end=end,
        frequency=frequency,
        date_format_value=date_format_value,
        verbose=verbose,
        session=session,
        raise_if_empty=True,
    )

    series_dfs = []
    for code, name in SERIES_CONFIG:
        df_ser = fetch_single_series(
            series_code=code,
            col_name=name,
            **common_kwargs,
        )
        series_dfs.append(df_ser)

    # Hepsini Tarih index'e göre join et
    out = series_dfs[0]
    for df_ser in series_dfs[1:]:
        out = out.join(df_ser, how="inner")

    return out.sort_index()


# =============================================================================
# 3) Net altın ton-sadece son haftalar
# =============================================================================

def compute_net_altin_ton(
    altin_swap: pd.DataFrame,
    start: None,
    end: Optional[str] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    EVDS haftalık resmi veriler + URDL'den altın swap serisi ile
    Net altın (ton) hesaplar.
    
    Beklenti:
      altin_swap.index : Tarih (haftalık, datetime)
      altin_swap.columns: ['altin_swap_milyar_usd']
    """
    df = fetch_weekly_official_from_evds(start= altin_swap.index.min().strftime("%d-%m-%Y") , end=end, verbose=verbose)

    # Altın swap ile inner join
    df = df.join(altin_swap, how="inner")

    # Ara hesaplar (gram cinsinden)
    df["Brut_altin_varlik"] = df["a11_gram"]

    df["Brut_altin_yukumluluk"] = (
        df["yurt_ici_mevduat_altin"]
        + df["hazine_altin"]
        + df["zorunlu_karsilik_altin"]
        + df["yurt_disi_bankalar_altin"]
    )

    df["Brut_altin_swap_gram"] = (
        df["a11_gram"]
        * df["altin_swap_milyar_usd"]
        / df["altin_rezerv_milyar_usd"]
    )

    df["Net_altin_ton"] = (
        df["Brut_altin_varlik"]
        - df["Brut_altin_yukumluluk"]
        + df["Brut_altin_swap_gram"]
        + df["a616_safi_olmayan_gram"]
    ) / 1_000_000  # gram -> ton

    return df[["Net_altin_ton"]].copy()




LBMA_GOLD_AM_URL = "https://prices.lbma.org.uk/json/gold_am.json"
TCMB_URDL_URL = (
    "https://www.tcmb.gov.tr/wps/wcm/connect/tr/tcmb+tr/main+menu/"
    "istatistikler/odemeler+dengesi+ve+ilgili+istatistikler/"
    "uluslararasi+rezervler+ve+doviz+likiditesi"
)
altin_swap = fetch_altin_swap_from_urdl()

df_net_altin = compute_net_altin_ton(
    altin_swap=altin_swap,        # URDL'den gelen seri
    start= altin_swap.index.min().strftime("%d-%m-%Y")      
)


last_date = df_net_altin.index.min()
gold_price = fetch_lbma_price(LBMA_GOLD_AM_URL)

merged = (
    gold_price
    # tarih kırpma
    .loc[lambda df: df.index >= last_date]
    .rename_axis("Tarih")
    .reset_index()
    # net altın ile birleştir
    .merge(
        df_net_altin.reset_index(),
        on="Tarih",
        how="outer"
    ).pipe(lambda df: df.loc[
        (df.index == df.index.max()) | df["Net_altin_ton"].notna()
    ])
    #.dropna(subset=["Net_altin_ton", "gold_valuation_price"])
    # USD değer ve ons haftalık değişim
    .assign(
        net_altin_usd=lambda d:
            d["Net_altin_ton"] * 32.1507466 * d["gold_valuation_price"] / 1_000_000,
        ons_price_change_weekly=lambda d:
            d["gold_valuation_price"].pct_change()
    )
    # değer değişim etkisi (bir önceki haftanın stok * haftalık fiyat değişimi)
    .assign(
        gold_valuation=lambda d:
            d["net_altin_usd"].shift(1) * d["ons_price_change_weekly"]
    )
)

merged.set_index("Tarih").to_excel("results/gold_valuation.xlsx")

